In [26]:
import pandas as pd
import sklearn 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

dataset_raw = pd.read_csv('train.csv', index_col=0)
dataset_raw

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500


In [27]:
Selected_data = dataset_raw[['LotArea','MSSubClass','LotConfig','LotShape','SalePrice']]
Selected_data

,LotArea,MSSubClass,LotConfig,LotShape,SalePrice
Id,,,,,
1,8450,60,Inside,Reg,208500
2,9600,20,FR2,Reg,181500
3,11250,60,Inside,IR1,223500
4,9550,70,Corner,IR1,140000
5,14260,60,FR2,IR1,250000
...,...,...,...,...,...
1456,7917,60,Inside,Reg,175000
1457,13175,20,Inside,Reg,210000
1458,9042,70,Inside,Reg,266500


In [28]:
missing_values = Selected_data.isnull().sum()
missing_values

LotArea       0
MSSubClass    0
LotConfig     0
LotShape      0
SalePrice     0
dtype: int64

In [29]:
X = Selected_data.drop(columns=['SalePrice']) 
y = Selected_data['SalePrice']
X_train_clean, X_test_clean, y_train_clean, y_test_clean = train_test_split(X, y, test_size=0.25, random_state=1)

In [30]:
categorical_columns = ['LotConfig','LotShape']  
numerical_columns = ['LotArea','MSSubClass'] 

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)
])

X_train = preprocessor.fit_transform(X_train_clean)
X_test = preprocessor.transform(X_test_clean)
y_train = y_train_clean
y_test = y_test_clean

In [31]:
from sklearn.linear_model import LinearRegression 
model = LinearRegression()  
model.fit(X_train, y_train) 

y_pred = model.predict(X_test)

def compute_rmse(y_test: np.ndarray, y_pred: np.ndarray, precision: int = 2) -> float:
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return round(rmse, precision)

rmse = compute_rmse(y_test, y_pred)
print(f"RMSE: {rmse}")

RMSE: 77557.84
